In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# 1. Mapping Label
sent_map   = {'Netral': 0, 'Positif': 1, 'Negatif': 2}
aspect_map = {'Pelayanan': 0, 'Kualitas': 1, 'Harga': 2}
inv_sent_map   = {v: k for k, v in sent_map.items()}
inv_aspect_map = {v: k for k, v in aspect_map.items()}

# 2. Dataset Class
class TokopediaDualDataset(Dataset):
    def __init__(self, texts, sent_labels, aspect_labels, tokenizer, max_length=128):
        self.texts = texts
        self.sent_labels = sent_labels
        self.aspect_labels = aspect_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'sentiment': torch.tensor(self.sent_labels[idx], dtype=torch.long),
            'aspect': torch.tensor(self.aspect_labels[idx], dtype=torch.long)
        }

# 3. Roberta-CNN Model
class RobertaCNNMultiTask(nn.Module):
    def __init__(self, num_sent_classes=3, num_aspect_classes=3):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        for param in self.roberta.parameters():
            param.requires_grad = False
        hidden_size = self.roberta.config.hidden_size
        self.conv1 = nn.Conv1d(in_channels=hidden_size, out_channels=256, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(0.3)
        self.sent_classifier = nn.Linear(256, num_sent_classes)
        self.aspect_classifier = nn.Linear(256, num_aspect_classes)

    def forward(self, input_ids, attention_mask):
        out = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = out.last_hidden_state.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        x = self.pool(x).squeeze(-1)
        x = self.dropout(x)
        sent_logits = self.sent_classifier(x)
        aspect_logits = self.aspect_classifier(x)
        return sent_logits, aspect_logits

# 4. Load Data
df = pd.read_csv("Tokopedia_preprocessed.csv")
texts = df["clean_content"].tolist()
sent_labels = [sent_map[s] for s in df["Sentimen"]]
aspect_labels = [aspect_map[a] for a in df["Aspek"]]

# 5. Split Data
train_texts, test_texts, train_sent, test_sent, train_aspect, test_aspect = train_test_split(
    texts, sent_labels, aspect_labels, test_size=0.2, random_state=42, stratify=sent_labels
)

# 6. Tokenizer dan DataLoader
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
train_dataset = TokopediaDualDataset(train_texts, train_sent, train_aspect, tokenizer)
test_dataset = TokopediaDualDataset(test_texts, test_sent, test_aspect, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# 7. Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaCNNMultiTask().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 8. Train Loop
for epoch in range(10):
    model.train()
    total_loss, correct_sent, correct_aspect = 0, 0, 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        sent_labels = batch["sentiment"].to(device)
        aspect_labels = batch["aspect"].to(device)

        optimizer.zero_grad()
        sent_logits, aspect_logits = model(input_ids, attention_mask)
        loss = criterion(sent_logits, sent_labels) + criterion(aspect_logits, aspect_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct_sent += (sent_logits.argmax(1) == sent_labels).sum().item()
        correct_aspect += (aspect_logits.argmax(1) == aspect_labels).sum().item()

    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f} - Sent Acc: {correct_sent/len(train_dataset):.4f} - Asp Acc: {correct_aspect/len(train_dataset):.4f}")

# 9. Evaluation
model.eval()
all_sent_preds, all_sent_trues = [], []
all_asp_preds, all_asp_trues = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        sent_labels = batch["sentiment"].to(device)
        aspect_labels = batch["aspect"].to(device)

        sent_logits, aspect_logits = model(input_ids, attention_mask)
        all_sent_preds.extend(sent_logits.argmax(1).cpu().tolist())
        all_sent_trues.extend(sent_labels.cpu().tolist())
        all_asp_preds.extend(aspect_logits.argmax(1).cpu().tolist())
        all_asp_trues.extend(aspect_labels.cpu().tolist())

# 10. Metrics
print("=== SENTIMEN CLASSIFICATION REPORT ===")
print(classification_report(all_sent_trues, all_sent_preds, target_names=[inv_sent_map[i] for i in range(3)]))

print("=== ASPEK CLASSIFICATION REPORT ===")
print(classification_report(all_asp_trues, all_asp_preds, target_names=[inv_aspect_map[i] for i in range(3)]))

print("=== CONFUSION MATRIX SENTIMEN ===")
print(confusion_matrix(all_sent_trues, all_sent_preds))

print("=== CONFUSION MATRIX ASPEK ===")
print(confusion_matrix(all_asp_trues, all_asp_preds))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 781.4462 - Sent Acc: 0.5038 - Asp Acc: 0.7714
Epoch 2 - Loss: 609.9183 - Sent Acc: 0.6341 - Asp Acc: 0.8286
Epoch 3 - Loss: 539.4896 - Sent Acc: 0.6786 - Asp Acc: 0.8545
Epoch 4 - Loss: 498.9988 - Sent Acc: 0.7079 - Asp Acc: 0.8685
Epoch 5 - Loss: 477.9194 - Sent Acc: 0.7235 - Asp Acc: 0.8709
Epoch 6 - Loss: 451.9039 - Sent Acc: 0.7418 - Asp Acc: 0.8804
Epoch 7 - Loss: 432.8797 - Sent Acc: 0.7554 - Asp Acc: 0.8858
Epoch 8 - Loss: 413.9715 - Sent Acc: 0.7690 - Asp Acc: 0.8909
Epoch 9 - Loss: 398.4756 - Sent Acc: 0.7750 - Asp Acc: 0.8988
Epoch 10 - Loss: 384.7100 - Sent Acc: 0.7874 - Asp Acc: 0.9009
=== SENTIMEN CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

      Netral       0.83      0.61      0.70       561
     Positif       0.88      0.88      0.88       767
     Negatif       0.71      0.87      0.78       672

    accuracy                           0.80      2000
   macro avg       0.81      0.79      0.79      2000
weighted avg  